[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_add_text.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Add text to PDF
Add text at a specified position on the first page of a
PDF document.

In [ ]:
import io
from pdftools_toolbox.pdf import Document, FileReference, Metadata, Page, PageCopyOptions, PageList
from pdftools_toolbox.geometry.real import Point
from pdftools_toolbox.pdf.content import Font, ContentGenerator, IccBasedColorSpace, Text, TextGenerator
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/BlankNoneNoTP.pdf'
input_file_path = 'BlankNoneNoTP.pdf'
prepare_file(input_url, input_file_path)
text_string = "Test String"
output_file_path = 'BlankWithText.pdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
def add_text(output_doc: Document, output_page: Page, text_string: str):
    # Create content generator and text object
    with ContentGenerator(output_page.content, False) as gen:
        text = Text.create(output_doc)

        # Create text generator
        with TextGenerator(text, font, font_size, None) as textGenerator:
            # Calculate position
            position = Point(border, output_page.size.height - border - font_size * font.ascent)

            # Move to position
            textGenerator.move_to(position)
            # Add given text string
            textGenerator.show_line(text_string)

        # Paint the positioned text
        gen.paint_text(text)

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Define global variables
    border = 40.0
    font_size = 15.0
    
    # Open input document
    with io.FileIO(input_file_path, 'rb') as in_stream:
        with Document.open(in_stream, None) as input_document:
    
            # Create output document
            with io.FileIO(output_file_path, 'wb+') as output_stream:
                with Document.create(output_stream, input_document.conformance, None) as output_document:
    
                    # Copy document-wide data
                    copy_document_data(input_document, output_document)
    
                    # Create a font
                    font = Font.create_from_system(output_document, "Arial", "Italic", True)
    
                    # Define page copy options
                    copy_options = PageCopyOptions()
    
                    # Copy first page, add text, and append to output document
                    out_page = Page.copy(output_document, input_document.pages[0], copy_options)
                    add_text(output_document, out_page, text_string)
                    output_document.pages.append(out_page)
    
                    # Copy remaining pages and append to output document
                    inPageRange = input_document.pages[1:]
                    copied_pages = PageList.copy(output_document, inPageRange, copy_options)
                    output_document.pages.extend(copied_pages)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")